# <div align="center"><a name="titulo"></a>Ocorrência de Câncer de Mama - UCI Machine Learning Repository</div>

<br>
<div align="center"><img src="https://i.imgur.com/71iwWHI.png"></div>
<hr>

<a name="indice"></a>

## Índice

1. [Introdução](#introducao)
2. [Preparação do Ambiente](#preparacao-ambiente)
3. [Schema UCI e Tabela Breast Cancer](#tabela-breast-cancer)
4. [Análise](#analise)
    1. [Categorização](#categorizacao)
    2. [Binarização](#binarizacao)
    3. [Codificação](#codificacao)
5. [Resultados](#resultados)

<a name="introducao"></a>

## 1. Introdução

Esta análise irá explorar o campo das ciências biológicas e da saúde, especificamente na **ocorrência de câncer de mama**, por meio dos dados disponibilizados pelo **Centro de Machine Learning e Sistemas Inteligentes da Universidade da California (UCI)**, localizada nos Estados Unidos. 

O **câncer de mama** se caracteriza por ser o tipo **mais incidente** em mulheres no mundo totalizando acima de **2.5 milhões de casos** representando **25% de novos casos** de novas ocorrências de câncer em mulheres segundo o *IARC* (2020). No Brasil não é diferente sendo **o tipo mais indicente em mulheres em todas as regiões do país**. Segundo estimativas do *INCA*, em 2022, é previsto em torno de **685 mil casos** da doença por aqui.


A pesquisa foi realizada pelo **Instituto de Oncologia do Centro Médico Universitário** de **Ljubljana** na ocasião da antiga Iugoslávia, hoje, capital da **Eslovênia**.

O dataset é composto por **286 ocorrências** de câncer de mama se dividindo em duas classes:
- **Recorrente**: possuindo 201 instâncias
- **Não recorrente**: 85 instâncias

E são descritos por 9 atributos, a seguir:
- **Idade**
- **Menopausa**: pré ou pós
- **Tamanho do Tumor**: em milímetros
- **Número de Nódulos**: 0 a 39
- **Ocorrência de Metastase**
- **Grau de Malignidade**: tendência de tumores se tornarem progressivamente piores. Grau 1 (menor) a 3 (maior)
- **Seio**: esquerdo ou direito
- **Quadrante de ocorrência**
- **Irradiação**: ocorrência ou não de tratamento por radioterapia

**Dataset em formato .names (pode ser aberto com Bloco de Notas. Pode-se renomear a extensão do arquivo para .csv) está disponível:**

 - [Projeto](breast-cancer.csv)
 - [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data)

**Dicionário de dados em formato .data (pode ser aberto com Bloco de Notas) está disponível:**
 - [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.names)
 - [UCI Machine Learning Repository](https://archive-beta.ics.uci.edu/ml/datasets/breast+cancer#Descriptive)

<br>

Zwitter, Matjaz & Soklic, Milan. (1988). Breast Cancer. UCI Machine Learning Repository.

<hr>


<a name="preparacao-ambiente"></a>

## 2. Preparação do Ambiente

[Início](#titulo)

In [ ]:
%pip install sqlalchemy
%pip install ipython-sql

In [2]:
#Bibliotecas

from sqlalchemy import create_engine
import sqlite3
import csv

# Conexão SQLite3: cria o banco de dados uci_ml.db
conexao = sqlite3.connect('uci_ml.db')
cursor = conexao.cursor()

# Conexão ipython-sql - execução SQL em tempo de execução
%reload_ext sql
%sql sqlite+pysqlite:///uci_ml.db

<hr>

## <a name="tabela-breast-cancer"></a>3. Tabela Breast Cancer

[Início](#titulo)

In [8]:
%%sql

/*Criação e estrutura da tabela tb_breast_cancer*/

CREATE TABLE IF NOT EXISTS TB_BREAST_CANCER (
  classe VARCHAR(100) NULL,
  idade VARCHAR(45) NULL,
  menopausa VARCHAR(45) NULL,
  tamanho_tumor VARCHAR(45) NULL,
  inv_nodes VARCHAR(45) NULL,
  node_caps VARCHAR(3) NULL,
  deg_malig INT NULL,
  seio VARCHAR(5) NULL,
  quadrante VARCHAR(10) NULL,
  irradiando VARCHAR(3) NULL);


 * sqlite+pysqlite:///uci_ml.db
Done.


[]

In [9]:
# Extração dos dados e inserção na tabela tb_breast_cancer
with open('breast-cancer.csv','r') as dataset:
    breast_cancer_data = csv.reader(dataset, delimiter=',',,)
    conexao.executemany('INSERT INTO TB_BREAST_CANCER VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',breast_cancer_data)
    conexao.commit()

<hr>

## <a name="analise"></a> 4. Análise

### <a name="tipo-variavel"></a>4.1. Tipos de Variáveis

Na Estatística Descritiva, em um modo sucinto, as variáveis podem ser classificadas em **qualitativas** ou **quantitativas**

As **qualitativas ou categóricas** expressam atributos que podem ser separados em diferentes categorias que se diferem por caraterísticas não numéricas. Podem ser:

1. **Nominais** em que não há uma ordenação específica: 
    - Gênero de uma pessoa (cisgênero/ transgênero/ não-binário)
    - Cor dos olhos (negros/ azuis/ verdes/ etc)
    - Categorias de filmes (terros/drama/ação/etc)
    - Etc

2. **Ordinais** quando existe certa ordem em seus valores
    - Nível de resfriamento de um congelador: baixo, médio, alto
    - Nível de aquecimento de um chuveiro: frio, morno e médio
    - Etc


As **quantitativas** são aquelas que podem ser medidas ou contabilizadas em termos de números. Elas podem ser 

1. **Discretas** quando resultam de um conjunto finito (enumerável, contável) de valores possíveis. Representam ***contagens***
    - Total de livros de uma biblioteca
    - Total de pessoas um evento
    - Quantidade de votos em uma eleição
    - Carros em um estacionamento

2. **Contínuas** quando resultam de um número infinito de valores possíveis que possam ser associados em uma escala contínua. São resultantes de ***medições***
    - Valores de criptomoedas
    - Altura de uma pessoa (m, cm, in, etc)
    - Peso de alimentos (kgf, gf, dgf)

[Início](#titulo)

### <a name="categorizacao"></a>4.2. Categorização

A **categorização** consiste em converter uma variável quantitativa para o tipo **categórico**. Um bom exemplo seria converter um atributo como a **idade** de uma pessoa (Ex: 20) para um **faixa etária** (Ex: 20 a 30 anos)

No dataset o atributo **tamanho_tumor** pode assumir estes possíveis valores de acordo com o dicionário de dados. A medição é em milímetros:

0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59

Como são 12 valores podemos realizar uma categorização para 6 conjuntos de valores (categorias):

 - **Muito Pequeno**: 0-4, 5-9
 - **Pequeno**: 10-14, 15-19
 - **Médio Baixo**: 20-24, 25-29
 - **Médio Grande**: 30-34, 35-39
 - **Grande**: 40-44, 45-49
 - **Muito Grande**: 50-54, 55-59

 Utilizando SQL temos:
 
 [Início](#titulo)

In [29]:
%%sql
/* Averiguar os registros */

SELECT DISTINCT
    tamanho_tumor AS 'Tamanho do Tumor (em mm)'
FROM
    tb_breast_cancer
ORDER BY tamanho_tumor ASC;

 * sqlite+pysqlite:///uci_ml.db
Done.


Tamanho do Tumor (em mm)
0-4
10-14
15-19
20-24
25-29
30-34
35-39
40-44
45-49
5-9


Podemos observar que não houve ocorrências de tumor com medidas de 55 a 59 mm o que caracterizaria como *Muito Grande*, porém há ocorrências de tamanho 50 a 54 mm por isso manteremos essa categoria. 

In [28]:
%%sql

/*Apenas valores distintos para exemplificação. Aplicação em todo dataset não utilizar o comando DISTINCT */

SELECT DISTINCT
    tamanho_tumor,
    CASE
        WHEN tamanho_tumor IN ('0-4' , '5-9') THEN 'Muito Pequeno'
        WHEN tamanho_tumor IN ('10-14' , '15-19') THEN 'Pequeno'
        WHEN tamanho_tumor IN ('20-24' , '25-29') THEN 'Médio Baixo'
        WHEN tamanho_tumor IN ('30-34' , '35-39') THEN 'Médio Alto'
        WHEN tamanho_tumor IN ('40-44' , '45-49') THEN 'Grande'
        WHEN tamanho_tumor IN ('50-54' , '55-59') THEN 'Muito Grande'
    END AS 'Categoria'
FROM
    tb_breast_cancer
ORDER BY tamanho_tumor ASC;

 * sqlite+pysqlite:///uci_ml.db
Done.


tamanho_tumor,Categoria
0-4,Muito Pequeno
10-14,Pequeno
15-19,Pequeno
20-24,Médio Baixo
25-29,Médio Baixo
30-34,Médio Alto
35-39,Médio Alto
40-44,Grande
45-49,Grande
5-9,Muito Pequeno


### <a name="binarizacao"></a>4.3. Binarização

Este técnica consistem em converter variáveis qualitativas em valores 0 ou 1. Normalmente é aplicado quando a variável assume apenas dois valores como *sim/não*, *verdadeiro/falso*, etc.

Podemos aplicar em nosso dataset no campo *classe* onde há apenas recorrência ou não-recorrência da doença.

[Início](#titulo)

In [27]:
%%sql

/* Apenas valores distintos para exemplificação. Aplicação em todo dataset não utilizar o comando DISTINCT */

SELECT DISTINCT classe,
    CASE
        WHEN classe = 'no-recurrence-events' THEN 0
        WHEN classe = 'recurrence-events' THEN 1
    END AS 'Recorrência'

FROM tb_breast_cancer

 * sqlite+pysqlite:///uci_ml.db
Done.


classe,Recorrência
no-recurrence-events,0
recurrence-events,1


### <a name="codificacao"></a>4.4. Codificação

A *codificação* é uma técnicas de conversão de dados categóricos para dados quantitativos por meio de uma representação numérica sem perder a essência da informação. Muito algoritmos de não conseguem processar dados que não sejam números. Por este motivo se faz necessária a codificação. A técnica pode se dar por duas formas.  


#### <a name="label-encoding"></a>4.4.1. Label-Encoding
Normalmente utilizado em variáveis que possuem poucas categorias (até 5 normalmente). Atribui um número para cada etiqueta (label/categoria) que a variável apresenta. Um exemplo em uma coluna com cores seria 1 para azul, 2 para amarelo, 3 para ver e assim por diante.

| Cor_Olhos | Cor_Label |
| :---: | :---: |
| Azul | 1 |
| Amarelo | 2 |
| Verde | 3 |

O estudo apresenta em sua variável ***Quadrante*** a localização dos nódulos cancerígenos em:
- **Left-Low (Esquerdo-Baixo)**
- **Left-Up (Esquerdo-Alto)**
- **Right-Up (Direito-Alto)**
- **Right-low (Direito-Baixo)**
- **Central (Centro)**

Podemos fazer o *label-encoding* atribuindo valores de 1 a 5 para cada uma das categorias. Para valores ausentes (NULL) atribui-se 0. 

Seguindo a ordem das etiquetas acima temos o código em SQL:

[Início](#titulo)

In [26]:
%%sql

/* Averiguar os registros */
SELECT DISTINCT
    quadrante
FROM
    tb_breast_cancer;

 * sqlite+pysqlite:///uci_ml.db
Done.


quadrante
left_low
right_up
left_up
right_low
central
?


In [25]:
%%sql

/* Apenas valores distintos para exemplificação. Aplicação em todo dataset não utilizar o comando DISTINCT */

SELECT DISTINCT quadrante,
    CASE
        WHEN quadrante = 'left_low' THEN 1
        WHEN quadrante = 'left_up' THEN 2
        WHEN quadrante = 'right_up' THEN 3
        WHEN quadrante = 'right_low' THEN 4
        WHEN quadrante = 'central' THEN 5
        ELSE 0
    END AS Quadrante
FROM
    tb_breast_cancer
ORDER BY
    quadrante;

 * sqlite+pysqlite:///uci_ml.db
Done.


quadrante,Quadrante
?,0
left_low,1
left_up,2
right_up,3
right_low,4
central,5


#### <a name="one-hot-encoding"></a> 4.4.2. One-Hot Encoding

O *Label-Encoding* possui um desvantagem já que em certos algoritmos de Machine Learning podem interpretar estes valores crescente como um hierarquia/ordem o que nem sempre é o desejado.
A resolução desta questão está no *One-Hot Encoding*. Esta prática cada etiqueta é convertida em uma nova coluna assumindo somente valores de 0 ou 1. É um caso especial de *binarização* como visto acima. Teria algo em torno deste aspecto:

| Cor_Olhos | Cor_Label_Azul | Cor_Label_Amarelo | Cor_Label_Verde |
| :---: | :---: | :---: | :---: |
| Azul | 1 | 0 | 0 |
| Amarelo | 0 | 1 | 0 |
| Verde | 0 | 0 | 1 |

O campo *Seio* traz informações sobre qual dos lados foi afetado, esquerdo ou direito. A codificação *One-Hot Encoding* assume esta forma:

[Início](#titulo)

In [24]:
%%sql 

/* Apenas valores distintos para exemplificação. Aplicação em todo dataset não utilizar o comando DISTINCT */

SELECT DISTINCT seio,
    CASE
        WHEN seio = 'left' then 1
        ELSE 0
    END as 'Seio_Label_Esquerdo',

    CASE
    WHEN seio = 'right' THEN 1
        ELSE 0
    END AS 'Seio_Label_Direito'

FROM
    tb_breast_cancer;


 * sqlite+pysqlite:///uci_ml.db
Done.


seio,Seio_Label_Esquerdo,Seio_Label_Direito
left,1,0
right,0,1


## <a name="resultados"></a>5. Resultados - Novo Dataset

A partir de todas as análises acima podemos criar um novo dataset que esteja adequado à análise e prática de Machine Learning de acordo com os parâmetros acima. Aplicaremos desta vez a todo dataset

[Início](#titulo)

In [ ]:
%%sql

SELECT 
    CASE
        WHEN classe = 'no-recurrence-events' THEN 0
        WHEN classe = 'recurrence-events' THEN 1
    END AS 'Recorrência',
    idade, menopausa,
    CASE
        WHEN tamanho_tumor IN ('0-4' , '5-9') THEN 'Muito Pequeno'
        WHEN tamanho_tumor IN ('10-14' , '15-19') THEN 'Pequeno'
        WHEN tamanho_tumor IN ('20-24' , '25-29') THEN 'Médio Baixo'
        WHEN tamanho_tumor IN ('30-34' , '35-39') THEN 'Médio Alto'
        WHEN tamanho_tumor IN ('40-44' , '45-49') THEN 'Grande'
        WHEN tamanho_tumor IN ('50-54' , '55-59') THEN 'Muito Grande'
    END AS 'Tamanho_Tumor',
    inv_nodes, node_caps, deg_malig,
    CASE
        WHEN seio = 'left' then 1
        ELSE 0
    END as 'Seio_Label_Esquerdo',
    CASE
    WHEN seio = 'right' THEN 1
        ELSE 0
    END AS 'Seio_Label_Direito',
    CASE
        WHEN quadrante = 'left_low' THEN 1
        WHEN quadrante = 'left_up' THEN 2
        WHEN quadrante = 'right_up' THEN 3
        WHEN quadrante = 'right_low' THEN 4
        WHEN quadrante = 'central' THEN 5
        ELSE 0
    END AS Quadrante,

    irradiando

FROM tb_breast_cancer

In [31]:
# Encerrando a conexão com o banco uci_ml.db
conexao.close()